# DAM

In [4]:
from fastai.vision.all import *
import pandas as pd
import cam
import util

In [5]:
base_model_name = 'densenet121-chexpert-final'

dls, labels = util.chexpert_data_loader(bs=8)

In [6]:
# Load the base model
model_base = cnn_learner(dls, densenet121, n_out=len(labels), path=Path('../saves/'), y_range=(0,1))
model_base.load(base_model_name)

# Create a new densenet121 model
model_dam = cnn_learner(dls, densenet121, n_out=len(labels), y_range=(0,1)).model

# Replace the body of new model with the body from the base model
base_model_body = model_base.model[0].state_dict()
model_dam[0].load_state_dict(base_model_body)

# Set all parameters to require grad
for param in model_dam.parameters():
    param.requires_grad = True
    
# Initialize DAM learner
dam_learner = util.DAM(model_dam, dls, folder=Path('../saves/densenet121-dam'), lr=0.03)

In [7]:
print(f'Initial AUC score: {dam_learner.eval()}')

Initial AUC score: 0.5075495764773205


In [5]:
dam_learner.train(max_epoch=5, lr_div=1.15)

epoch: 0, train_loss: -0.021034, train_auc:0.705130, test_auc:0.682756, lr:0.050000
Update Regularizer @ T=22364!
epoch: 1, train_loss: -0.002383, train_auc:0.672174, test_auc:0.807442, lr:0.040000
Update Regularizer @ T=22364!
epoch: 2, train_loss: -0.006048, train_auc:0.709431, test_auc:0.796139, lr:0.032000
Update Regularizer @ T=22364!
epoch: 3, train_loss: -0.001118, train_auc:0.704700, test_auc:0.800208, lr:0.025600
Update Regularizer @ T=22364!


Get the categorywise AUC scores of the best model.

In [6]:
dam_learner.load('m-epoch 2-2021_May_09-01_59_24.pth.tar')

In [ ]:
print(f'Categorywise AUC score: {dam_learner.eval(average=None)}')